<center>
    <h1> ILI-286 - Laboratorio #5 </h1>
    <h2> Sistemas de reacción-difusión </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Reacción-Difusión y formación de patrones de Turing](#se1)
  2. [Diferencias Finitas en 2D](#se2)
  3. [Método de las líneas](#se3)
  4. [Simulaciones](#se4)
* [Conclusiones](#Conclusiones)
* [Referencias](#Referencias)
* [Anexos](#Anexos) $\leftarrow$ Comenzar ejecución aquí
  * [Métodos implementados](#an1)
  * [Utilidades](#an5)

<div id='intro' />
## Introducción
En este laboratorio se analizará el sistema de reacción-difusión de Alan Turing que propone un modelo
para la formación de pigmentación en animales, particularmente se trabajará con el modelo propuesto por 
FitzHugh-Nagumo donde interactuan dos ecuaciones diferenciales tanto espacialmente como en el tiempo.
Se resolverá el problema por medio de diferencias finitas (tanto con discretizaciones backward como forward)
y por el método de lineas, con estos resultados se estimará la incidencia de los parametros que afectan al
sistema y los límites propios de los algoritmos.

<div id='desarrollo'/>
## Desarrollo y analisís de resultados

<div id='se1'/>
## 1.- Reacción-Difusión y formación de patrones de Turing

El sistema queda definido por:
\begin{align}
    \frac{\partial u}{\partial t} &= 
            D_u \, (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) +
            \lambda \, u - u^3 - \kappa - \sigma v \\
    \tau \, \frac{\partial v}{\partial t} &= 
            D_v \, (\frac{\partial^2 v}{\partial x^2} + 
            \frac{\partial^2 v}{\partial y^2}) + u - v
\end{align}
Se trabajará con condiciones de borde periodicas.

<div id='se2'/>
## 2.- Diferencias finitas en 2D

### a) Propuesta de discretización "Forward"
Utilizando las siguientes aproximaciones:
\begin{align}
    \frac{\partial^2 u}{\partial x^2} &= \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2}\\
    \frac{\partial^2 u}{\partial y^2} &= \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2}\\
    \frac{\partial u}{\partial t} &= \frac{u(x,y,t+\Delta t) - u(x,y,t)}{\Delta t}
\end{align}

Análogo para $v(x,y,t)$.

Las ecuaciones quedan como:
\begin{align}
    \frac{u(x,y,t+\Delta t) - u(x,y,t)}{\Delta t} =& \,
        D_u \, \left(
            \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} + 
            \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} 
        \right)\\
        &+ \lambda \, u(x,y,t) - u(x,y,t)^3 - \kappa - \sigma v(x,y,t)\\
    &\\
    \tau \, \frac{v(x,y,t+\Delta t) - v(x,y,t)}{\Delta t} =&\,
        D_v \, \left(
            \frac{v(x+\Delta x,y,t)-2v(x,y,t)+v(x-\Delta x,y,t)}{\Delta x^2} + 
            \frac{v(x,y+\Delta y,t)-2v(x,y,t)+v(x,y-\Delta y,t)}{\Delta y^2} 
        \right)\\
        &+ u(x,y,t) - v(x,y,t)
\end{align}

Ahora si reemplazamos $w_{i,j,k} = u(x,y,t)$ y $ b_{i,j,k} = v(x,y,t) $, donde:
\begin{matrix}
    x= x_0+i \,\Delta x &\Longrightarrow &i= \frac{x-x_0}{\Delta x}\\
    y= y_0+j \,\Delta y &\Longrightarrow &j= \frac{y-y_0}{\Delta y}\\
    t= t_0+k \,\Delta t &\Longrightarrow &k= \frac{t-t_0}{\Delta t}
\end{matrix}

Finalmente, obtenemos:

\begin{align}
    \frac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t} &=
        D_u  \, \left(
            \frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} +
            \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k} \\
    \tau \, \frac{b_{i,j,k+1} - b_{i,j,k}}{\Delta t} &=
        D_v \, \left(
            \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} + 
            \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right)
        + w_{i,j,k} - b_{i,j,k}
\end{align}

Despejando para el tiempo $k+1$ nos queda:
\begin{align}
    w_{i,j,k+1} &=
        \Delta t \left(D_u  \, \left(
            \frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} + 
            \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k} \right ) + w_{i,j,k} \\
    b_{i,j,k+1} &= 
        \frac{\Delta t}{\tau} \left(D_v \, \left(
            \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} +
            \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right)
        + w_{i,j,k} - b_{i,j,k} \right ) + b_{i,j,k}
\end{align}

El error de aproximación es de: $O(\Delta t) + O(\Delta x^2) + O(\Delta y^2)$

### b) Implementación de esquema explícito "Forward"

In [33]:
def solve_forward(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa):
    u0 = np.vstack((u0, u0[1,:]))
    u0 = np.hstack((u0, u0[:,[1]]))
    v0 = np.vstack((v0, v0[1,:]))
    v0 = np.hstack((v0, v0[:,[1]]))
    Ny = u0.shape[0];
    Nx = u0.shape[1];
    Nt = (int)(np.ceil(Tf/dt))+1;
    w = np.zeros([Nt, Nx, Ny])
    b = np.zeros([Nt, Nx, Ny])
    w[0,:,:] = u0;
    b[0,:,:] = v0;
    for k in range (Nt-1):
        u_xx = (np.roll(w[k,:,:], 1, axis=1) - 2 * w[k,:,:] + np.roll(w[k,:,:], -1, axis=1))/dx**2;
        u_yy = (np.roll(w[k,:,:], 1, axis=0) - 2 * w[k,:,:] + np.roll(w[k,:,:], -1, axis=0))/dy**2;
        u_laplaciano = u_xx + u_yy;
        
        v_xx = (np.roll(b[k, :, :], 1, axis=1) - 2 * b[k,:,:] + np.roll(b[k,:,:], -1, axis=1))/dx**2;
        v_yy = (np.roll(b[k, :, :], 1, axis=0) - 2 * b[k,:,:] + np.roll(b[k,:,:], -1, axis=0))/dy**2;
        v_laplaciano = v_xx + v_yy
        
        w[k+1,:,:] = dt * (Du * u_laplaciano + lamb * w[k,:,:] - np.power(w[k,:,:],3) - kappa - sigma * b[k,:,:]) + w[k,:,:];
        
        b[k+1,:,:] = (dt/tau) * (Dv * v_laplaciano + w[k,:,:] - b[k,:,:]) + b[k,:,:];
    return w[:, 0:Nx-2, 0:Nx-2];
    #return w;

#u0 = np.array([[[0.1,0.1,0.2,0.3],[0.2,0.1,0.3,0.1],[0.3,0.1,0.1,0.2],[0.2,0.3,0.4,0.5]],[[0.1,0.2,0.1,0.2],[0.1,0.2,0.3,0.1],[0.1,0.1,0.1,0.3],[0.1,0.3,0.2,0.4]]]);
#v0 = np.array([[[0.1,0.2,0.1,0.2],[0.1,0.2,0.3,0.1],[0.1,0.1,0.1,0.3],[0.1,0.3,0.2,0.4]],[[0.1,0.1,0.2,0.3],[0.2,0.1,0.3,0.1],[0.3,0.1,0.1,0.2],[0.2,0.3,0.4,0.5]]]);
#print(u0[:,0,:])
#print(np.concatenate((u0,u0[:,0,:]), axis=1,3))
#print(np.concatenate((u0,u0[:,:,0]), axis=0))

#u0 = np.matrix([[0.1,0.1,0.2,0.3],[0.2,0.1,0.3,0.1],[0.3,0.1,0.1,0.2],[0.2,0.3,0.4,0.5]]);
#v0 = np.matrix([[0.1,0.2,0.1,0.2],[0.1,0.2,0.3,0.1],[0.1,0.1,0.1,0.3],[0.1,0.3,0.2,0.4]]);
#print(np.concatenate((u0,u0[1,:]), axis=0))
#print(np.concatenate((u0,u0[:,1]), axis=1))
                        
                        
#dx = 1;
#dy = 1;
#dt = 1;
#tf = 5;
#Du = 2;
#Dv = 2;
#lamb = 1;
#tau = 1;
#sigma = 1;
#kappa = 1;
#print(u0);
#print(v0);

#print(solve_forward(u0, v0, dx, dy, (dx**2)/10, tf, Du, Dv, lamb, tau, sigma, kappa))

### c) Propuesta de discretización "Backward"
Aproximaciones:
\begin{align}
    \frac{\partial^2 u}{\partial x^2} &= \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2}\\
    \frac{\partial^2 u}{\partial y^2} &= \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2}\\
    \frac{\partial u}{\partial t} &= \frac{u(x,y,t) - u(x,y,t-\Delta t)}{\Delta t}
\end{align}

Análogo para $v(x,y,t)$.

Las ecuaciones quedan como:
\begin{align}
    \frac{u(x,y,t) - u(x,y,t - \Delta t)}{\Delta t} =&\,
        D_u \, \left(
            \frac{u(x+\Delta x, y, t)-2u(x,y,t)+u(x-\Delta x,y,t)}{\Delta x^2} + 
            \frac{u(x, y+\Delta y, t)-2u(x,y,t)+u(x,y-\Delta y,t)}{\Delta y^2} \right)\\
        &+ \lambda \, u(x,y,t) - u(x,y,t)^3 - \kappa - \sigma v(x,y,t)\\
        &\\
    \tau \, \frac{v(x,y,t) - v(x,y,t-\Delta t)}{\Delta t} =&\,
        D_v \, \left(
            \frac{v(x+\Delta x,y,t)-2v(x,y,t)+v(x-\Delta x,y,t)}{\Delta x^2} +
            \frac{v(x,y+\Delta y,t)-2v(x,y,t)+v(x,y-\Delta y,t)}{\Delta y^2} \right)\\
        &+ u(x,y,t) - v(x,y,t)
\end{align}

Ahora si reemplazamos $w_{i,j,k} = u(x,y,t)$ y $ b_{i,j,k} = v(x,y,t) $, donde:
\begin{matrix}
    x= x_0+i \,\Delta x &\Longrightarrow &i= \frac{x-x_0}{\Delta x}\\
    y= y_0+j \,\Delta y &\Longrightarrow &j= \frac{y-y_0}{\Delta y}\\
    t= t_0+k \,\Delta t &\Longrightarrow &k= \frac{t-t_0}{\Delta t}
\end{matrix}

Finalmente, obtenemos:
\begin{align}
    \frac{w_{i,j,k} - w_{i,j,k-1}}{\Delta t} &=
        D_u  \, \left(
            \frac{w_{i+1, j, k}-2w_{i,j,k}+w_{i-1,j,k}}{\Delta x^2} +
            \frac{w_{i, j+1, k}-2w_{i,j,k}+w_{i,j-1,k}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma b_{i,j,k}\\
    \tau \, \frac{b_{i,j,k} - b_{i,j,k-1}}{\Delta t} &=
        D_v \, \left(
            \frac{b_{i+1,j,k}-2b_{i,j,k}+b_{i-1,j,k}}{\Delta x^2} + 
            \frac{b_{i,j+1,k}-2b_{i,j,k}+b_{i,j-1,k}}{\Delta y^2} \right)
        + w_{i,j,k} - b_{i,j,k}
\end{align}

Cambiando $k$ por $k+1$:
\begin{align}
    \frac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t} &=
        D_u  \, \left(
            \frac{w_{i+1, j, k+1}-2w_{i,j,k+1}+w_{i-1,j,k+1}}{\Delta x^2} +
            \frac{w_{i, j+1, k+1}-2w_{i,j,k+1}+w_{i,j-1,k+1}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k+1} - w_{i,j,k+1}^3 - \kappa - \sigma b_{i,j,k+1}\\
    \tau \, \frac{b_{i,j,k+1} - b_{i,j,k}}{\Delta t} &=
        D_v \, \left(
            \frac{b_{i+1,j,k+1}-2b_{i,j,k+1}+b_{i-1,j,k+1}}{\Delta x^2} +
            \frac{b_{i,j+1,k+1}-2b_{i,j,k+1}+b_{i,j-1,k+1}}{\Delta y^2} \right)
        + w_{i,j,k+1} - b_{i,j,k+1}
\end{align}        

Despejando para el tiempo $k+1$:

\begin{align}
    w_{i,j,k} &= 
        w_{i,j,k+1} - \Delta t \left ( D_u  \, \left(
            \frac{w_{i+1, j, k+1}-2w_{i,j,k+1}+w_{i-1,j,k+1}}{\Delta x^2} +
            \frac{w_{i, j+1, k+1}-2w_{i,j,k+1}+w_{i,j-1,k+1}}{\Delta y^2} \right)
        + \lambda \, w_{i,j,k+1} - w_{i,j,k+1}^3 - \kappa - \sigma b_{i,j,k+1} \right )\\
    b_{i,j,k} &= 
        b_{i,j,k+1} - \frac{\Delta t}{\tau} \left( D_v \, \left(
            \frac{b_{i+1,j,k+1}-2b_{i,j,k+1}+b_{i-1,j,k+1}}{\Delta x^2} +
            \frac{b_{i,j+1,k+1}-2b_{i,j,k+1}+b_{i,j-1,k+1}}{\Delta y^2} \right)
        + w_{i,j,k+1} - b_{i,j,k+1} \right )
\end{align}

Agrupando incógnitas en la primera ecuación
\begin{align}
    w_{i,j,k} =&\,
        w_{i,j,k+1} - \frac{\Delta t \, D_u \, w_{i+1, j, k+1}}{\Delta x^2} + 
        2\frac{\Delta t \, D_u \, w_{i,j,k+1}}{\Delta x^2} - 
        \frac{\Delta t \, D_u \, w_{i-1,j,k+1}}{\Delta x^2} - 
        \frac{\Delta t \, D_u \, w_{i, j+1, k+1}}{\Delta y^2} +
        2 \frac{\Delta t \, D_u \, w_{i,j,k+1}}{\Delta y^2} -
        \frac{\Delta t \, D_u \, w_{i,j-1,k+1}}{\Delta y^2}\\
        &- \Delta t \, \lambda \, w_{i,j,k+1} + \Delta t \, w_{i,j,k+1}^3 + 
        \Delta t \, \kappa + \Delta t \, \sigma b_{i,j,k+1}\\
    &\\
     w_{i,j,k} - \Delta t \, \kappa =&\,
         (1+ 2\frac{\Delta t \, D_u}{\Delta x^2} + 
         2 \frac{\Delta t \, D_u}{\Delta y^2 } - 
         \Delta t \, \lambda + \Delta t \, w_{i,j,k+1}^2) \, w_{i,j,k+1} -
         \frac{\Delta t \, D_u \, w_{i+1, j, k+1}}{\Delta x^2} -
         \frac{\Delta t \, D_u \, w_{i-1,j,k+1}}{\Delta x^2} -
         \frac{\Delta t \, D_u \, w_{i, j+1, k+1}}{\Delta y^2}
         - \frac{\Delta t \, D_u \, w_{i,j-1,k+1}}{\Delta y^2} \\
         &+ \Delta t \, \sigma b_{i,j,k+1}\\
\end{align}

Cambiando $ \delta_u = \Delta t \, D_u $:

$$ (1+ 2\frac{\delta_u}{\Delta x^2} + 2 \frac{\delta_u}{\Delta y^2 } - \frac{\delta_u}{D_u}  \, \lambda + \frac{\delta_u}{D_u} \, w_{i,j,k+1}^2) \, w_{i,j,k+1} - \frac{\delta_u \, w_{i+1, j, k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i-1,j,k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i, j+1, k+1}}{\Delta y^2} - \frac{\delta_u \, w_{i,j-1,k+1}}{\Delta y^2} + \frac{\delta_u}{D_u} \, \sigma b_{i,j,k+1} = w_{i,j,k} - \frac{\delta_u}{D_u} \, \kappa $$


Agrupando incógnitas en la segunda ecuación
\begin{align}
     b_{i,j,k} &=
         b_{i,j,k+1} - \frac{\Delta t \, D_v \, b_{i+1,j,k+1}}{\Delta x^2 \, \tau} +
         2 \frac{\Delta t \, D_v \, b_{i,j,k+1}}{\Delta x^2 \, \tau} -
         \frac{\Delta t \, D_v \, b_{i-1,j,k+1}}{\Delta x^2 \, \tau} -
         \frac{\Delta t \, D_v \, b_{i,j+1,k+1}}{\Delta y^2 \, \tau} +
         2 \frac{\Delta t \, D_v \, b_{i,j,k+1}}{\Delta y^2 \, \tau} -
         \frac{\Delta t \, D_v \, b_{i,j-1,k+1}}{\Delta y^2 \, \tau} -
         \frac{\Delta t}{\tau} \, w_{i,j,k+1} + \frac{\Delta t \, b_{i,j,k+1}}{\tau}\\
     b_{i,j,k} &=
         (1+ 2 \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} +
         2 \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} +
         \frac{\Delta t}{\tau}) \, b_{i,j,k+1} -
         \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} \, b_{i+1,j,k+1} -
         \frac{\Delta t \, D_v}{\Delta x^2 \, \tau} \, b_{i-1,j,k+1} -
         \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} \, b_{i,j+1,k+1} -
         \frac{\Delta t \, D_v}{\Delta y^2 \, \tau} \, b_{i,j-1,k+1} -
         \frac{\Delta t}{\tau} \, w_{i,j,k+1}
\end{align}

Cambiando $ \delta_v = \displaystyle \frac{\Delta t \, D_v}{\tau} $

$$ (1+ 2 \frac{\delta_v}{\Delta x^2} + 2 \frac{\delta_v}{\Delta y^2} + \frac{\delta_v}{D_v}) \, b_{i,j,k+1} - \frac{\delta_v}{\Delta x^2} \, b_{i+1,j,k+1}   - \frac{\delta_v}{\Delta x^2} \, b_{i-1,j,k+1} - \frac{\delta_v}{\Delta y^2} \, b_{i,j+1,k+1}  - \frac{\delta_v}{\Delta y^2} \, b_{i,j-1,k+1} - \frac{\delta_v}{D_v} \, w_{i,j,k+1} = b_{i,j,k} $$

El error de aproximación es de: $O(\Delta t) + O(\Delta x^2) + O(\Delta y^2)$

En resumen, usando $ \delta_u = \displaystyle \Delta t \, D_u $ y $ \delta_v = \displaystyle \frac{\Delta t \, D_v}{\tau} $ nos queda finalmente el siguiente sistema de ecuaciones no-lineales:

\begin{align}
    (1+ 2\frac{\delta_u}{\Delta x^2} + 2 \frac{\delta_u}{\Delta y^2 } -
    \frac{\delta_u}{D_u}  \, \lambda + \frac{\delta_u}{D_u} \, w_{i,j,k+1}^2) \, w_{i,j,k+1} -
    \frac{\delta_u \, w_{i+1, j, k+1}}{\Delta x^2} - \frac{\delta_u \, w_{i-1,j,k+1}}{\Delta x^2} -
    \frac{\delta_u \, w_{i, j+1, k+1}}{\Delta y^2} - \frac{\delta_u \, w_{i,j-1,k+1}}{\Delta y^2} +
    \frac{\delta_u}{D_u} \, \sigma b_{i,j,k+1} &= w_{i,j,k} - \frac{\delta_u}{D_u} \, \kappa \\
    (1+ 2 \frac{\delta_v}{\Delta x^2} + 2 \frac{\delta_v}{\Delta y^2} +
    \frac{\delta_v}{D_v}) \, b_{i,j,k+1} - \frac{\delta_v}{\Delta x^2} \, b_{i+1,j,k+1} -
    \frac{\delta_v}{\Delta x^2} \, b_{i-1,j,k+1} - \frac{\delta_v}{\Delta y^2} \, b_{i,j+1,k+1} -
    \frac{\delta_v}{\Delta y^2} \, b_{i,j-1,k+1} - \frac{\delta_v}{D_v} \, w_{i,j,k+1} &= b_{i,j,k}
\end{align}

### d) Implementación de esquema implícito "Backward"

In [5]:
def solve_backward(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa):
    Ny = u0.shape[0];
    Nx = u0.shape[1];
    Nt = (int)(np.ceil(Tf/dt));
    w = np.zeros([Nt, Nx, Ny])
    b = np.zeros([Nt, Nx, Ny])
    w[0,:,:] = u0;
    b[0,:,:] = v0;
    
    delta_u = dt*Du;
    delta_v = dt*Dv/tau;
    cons1 = 1+2*delta_u / dx**2 +2 * delta_u/dy**2-delta_u*lamb/Du;
    cons2 = 1+2*delta_v/dx**2+2*delta_v/dy**2+delta_v/Dv;
    for k in range (Nt-1):
        print("k=")
        print(k)
        def func(wbk):
            wbk = wbk.reshape(2*Nx, Ny);
            wk = wbk[0:Nx,:];
            bk = wbk[Nx:2*Nx,:];
            # b[0], w[0] => i  , j-1
            # b[1], w[1] => i-1, j
            # b[2], w[2] => i  , j
            # b[3], w[3] => i+1, j
            # b[4], w[4] => i  , j+1
    #        return np.array([(1+2*delta_u / dx**2 +2 * delta_u/dy**2-delta_u*lamb/Du + (delta_u*w[2]**2)/Du)*w[2]
    #                    -delta_u*w[3]/dx**2-delta_u*w[1]/dx**2-delta_u*w[4]/dy**2-delta_u*w[0]/dy**2
    #                    +delta_u*sigma*b[2]/Du + delta_u*kappa/Du - w_ijk
    #                     ,
    #                     (1+2*delta_v/dx**2+2*delta_v/dy**2+delta_v/Dv)*b[2]-delta_v*b[3]/dx**2
    #                     -delta_v*b[1]/dx**2-delta_v*b[4]/dy**2-delta_v*b[0]/dy**2-delta_v*w[2]/Dv
    #                     -b_ijk
    #                    ]);
            return np.array([np.multiply( cons1 + (delta_u*np.power(wk[:,:],2))/Du, wk[:,:])
                    -delta_u*np.roll(wk[:,:], -1, axis=1)/dx**2-delta_u*np.roll(wk[:,:], 1, axis=1)
                         /dx**2-delta_u*np.roll(wk[:,:], -1, axis=0)/dy**2-delta_u*np.roll(wk[:,:], 1, axis=0)/dy**2
                    +delta_u*sigma*bk[:,:]/Du + delta_u*kappa/Du - w[k,:,:]
                     ,
                     cons2*bk[:,:]-delta_v*np.roll(bk[:,:], -1, axis=1)/dx**2
                     -delta_v*np.roll(bk[:,:], 1, axis=1)/dx**2-delta_v*np.roll(bk[:,:], -1, axis=0)
                         /dy**2-delta_v*np.roll(bk[:,:], 1, axis=0)/dy**2-delta_v*wk[:,:]/Dv
                     -b[k,:,:]
                    ]).ravel();
        wk0 = np.ones([Nx, Ny]);
        #wk0 = np.matrix ([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]);
        bk0 = wk0;
        #[w[k+1,:,:], b[k+1,:,:]] = sp.optimize.fsolve(func, [wk0, bk0]);
        #print("fsolve")
        temp = sp.optimize.fsolve(func, np.array([wk0, bk0]).ravel()).reshape(2,Nx,Ny);
        #print(temp)
        w[k+1,:,:] = temp[0,:,:];
        b[k+1,:,:] = temp[1,:,:];
    return w;

<div id='se3'/>
## 3.- Método de las líneas

### a) Discretización y formulación del sistema

Tenemos:
\begin{align}
    \frac{\partial u}{\partial t} &= 
            D_u \, (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) +
            \lambda \, u - u^3 - \kappa - \sigma v \\
    \tau \, \frac{\partial v}{\partial t} &= 
            D_v \, (\frac{\partial^2 v}{\partial x^2} + 
            \frac{\partial^2 v}{\partial y^2}) + u - v
\end{align}

utilizando diferencias centradas para discretizar el dominio espacial nos queda:

\begin{align}
    \frac{\partial u(x_i,y_j,t)}{\partial t} =&\,
            D_u \left(
                \frac{u(x_{i+1},y_j,t) - 2 u(x_i,y_j,t) + u(x_{i-1},y_j,t)}{\Delta x^2} + 
                \frac{u(x_i,y_{j+1},t) - 2 u(x_i,y_j,t) + u(x_i,y_{j-1},t)}{\Delta y^2}
            \right)\\
            & + \lambda u(x_i,y_j,t) - (u(x_i,y_j,t))^3 - \kappa - \sigma v(x_i,y_j,t) \\
    &\\
    \tau \, \frac{\partial v(x_i,y_j,t)}{\partial t} =&\,
            D_v \left(
                \frac{v(x_{i+1},y_j,t) - 2 v(x_i,y_j,t) + v(x_{i-1},y_j,t)}{\Delta x^2} + 
                \frac{v(x_i,y_{j+1},t) - 2 v(x_i,y_j,t) + v(x_i,y_{j-1},t)}{\Delta y^2}
            \right)\\
            & + u(x_i,y_j,t) - v(x_i,y_j,t)
\end{align}

como podemos notar que ahora el problema diferencial es solo dependiente del tiempo. Hacemos el cambio de variable $u(x_i, y_j, y) = w_{i,j}(t)$ y $v(x_i, y_j, y) = b_{i,j}(t)$.
Así, el sistema dinámico acoplado resultante queda determinado por las siguientes ecuaciones:

\begin{align}
    \dot{w_{ij}}&=
        D_u \left( 
            \frac{w_{i+1,j} - 2 w_{i,j} + w_{i-1,j}}{\Delta x^2} + 
            \frac{w_{i,j+1} - 2 w_{i,j} + w_{i,j-1}}{\Delta y^2} \right) 
        + \lambda w_{i,j} - w^3_{i,j} - \kappa - \sigma b_{i,j} \\
    \dot{b_{i,j}}&= 
        \frac{D_v}{\tau} \left(
            \frac{b_{i+1,j} - 2 b_{i,j} + b_{i-1,j}}{\Delta x^2} + 
            \frac{b_{i,j+1} - 2 b_{i,j} + b_{i,j-1}}{\Delta y^2} \right) 
        + \frac{w_{i,j} - b_{i,j}}{\tau}
\end{align}

Ordenando convenientemente tenemos:
\begin{align}
    \dot{w_{i,j}} &=
        \frac{D_u}{\Delta x^2}(w_{i-1,j}+w_{i+1,j}) + 
        \frac{D_u}{\Delta y^2}(w_{i,j-1}+w_{i,j+1}) -
        (2D_u(x^{-2}+y^{-2}) + \lambda)w_{i,j} - w_{i,j}^3
        +\kappa -\sigma b_{i,j} \\
    \dot{b_{i,j}} &=
        \frac{D_v}{\tau \Delta x^2}(b_{i-1,j}+b_{i+1,j}) +
        \frac{D_v}{\tau \Delta y^2}(b_{i,j-1}+b_{i,j+1}) -
        \frac{2D_v}{\tau}(x^{-2}+y^{-2})b_{i,j} - \frac{w_{i,j}+b_{i,j}}{\tau}
\end{align}

### b) Implementación

In [6]:
def solve_mol(u0, v0, dx, dy, dt, Tf, Du, Dv, lamb, tau, sigma, kappa, method='RK4'):
    if method not in ['RK4', 'RK2']:
        print("El método debe ser 'RK2' o 'RK4'")
        return None
    Ny, Nx, Nt = u0.shape[0], u0.shape[1], int(np.ceil(Tf/dt))
    w = np.zeros([Nt, Nx, Ny])
    b = np.zeros([Nt, Nx, Ny])
    w[0,:,:] = u0;
    b[0,:,:] = v0;
    #Algunos calculos que no varian con el tiempo:
    h2   = dt/2
    Dux  = Du/(dx**2)
    Duy  = Du/(dy**2)
    Duxy = -2*Du*((dx**-2) + (dy**-2)) + lamb
    Dvx  = Dv/(tau*(dx**2))
    Dvy  = Dv/(tau*(dy**2))
    Dvxy = (-2*Dv/tau)*((dx**-2) + (dy**-2))
    #Como las ecuaciones son autonomas se utilizará t como subindice:
    def U(t, xy): #t:subindice del tiempo, xy:matriz de 2 dimenciones con los valores en x e y.
        return Dux*(np.roll(xy, -1, axis=1) + np.roll(xy, 1, axis=1)) +\
               Duy*(np.roll(xy, -1, axis=0) + np.roll(xy, 1, axis=0)) +\
               Duxy*xy - xy**3 - sigma*b[t,:,:] + kappa
    def V(t, xy): #igual que la anterior pero para V
        return Dvx*(np.roll(xy, -1, axis=1) + np.roll(xy, 1, axis=1)) +\
               Dvy*(np.roll(xy, -1, axis=0) + np.roll(xy, 1, axis=0)) +\
               Dvxy*xy - (w[t,:,:] - xy)/tau
    #Ignorando los cambios en t a proposito (no cambia nada por que la ec es autonoma).
    if method=='RK2':
        for k in range(Nt-1):
            ku1 = w[k,:,:] + h2*U(k, w[k,:,:])
            w[k+1,:,:] = w[k,:,:] + dt*U(k, ku1)
            kv1 = b[k,:,:] + h2*V(k, b[k,:,:])
            b[k+1,:,:] = b[k,:,:] + dt*V(k, kv1)
    else:
        for k in range(Nt-1):
            ku1 = U(k, w[k,:,:])
            ku2 = U(k, w[k,:,:] + h2*ku1)
            ku3 = U(k, w[k,:,:] + h2*ku2)
            ku4 = U(k, w[k,:,:] + dt*ku3)
            w[k+1,:,:] = w[k,:,:] + (dt/6)*(ku1 + 2*(ku2+ku3) + ku4)
            kv1 = V(k, b[k,:,:])
            kv2 = V(k, b[k,:,:] + h2*kv1)
            kv3 = V(k, b[k,:,:] + h2*kv2)
            kv4 = V(k, b[k,:,:] + dt*kv3)
            b[k+1,:,:] = b[k,:,:] + (dt/6)*(kv1 + 2*(kv2+kv3) + kv4)
    return w

<div id='se4'\>
## 4.- Simulaciones
Se establecen las variables a usar:

In [5]:
#Generando puntos de la malla aleatoria
Nx, Ny = 50, 50
u0 = np.random.rand(Nx,Ny)
u0[Nx-1,:] = u0[0,:]
u0[:,Ny-1] = u0[:,0]
v0 = np.random.rand(Nx,Ny)
v0[Nx-1,:] = v0[0,:]
v0[:,Ny-1] = v0[:,0]
dx, dy = 1/(Nx-1), 1/(Ny-1)
tf   = 10

### a) Significado de los parametros

### b) Ejecución de las implementaciones

In [32]:
#Parametros suministrados.
Du = 3e-4
Dv = 4e-3
lamb = 1.0
tau  = 0.2
sigm = 1.0
kapp = -0.004

q = np.array([[1,2,3,4],[3,4,5,6],[1,2,3,4]]);
print(q);
print(q[:,[1]])
print(np.hstack((q, q[:,[1]])))

[[1 2 3 4]
 [3 4 5 6]
 [1 2 3 4]]
[[2]
 [4]
 [2]]
[[1 2 3 4 2]
 [3 4 5 6 4]
 [1 2 3 4 2]]


In [34]:
A = solve_forward(u0, v0, dx, dy, dx**2, tf, Du, Dv, lamb, tau, sigm, kapp)
print(A)

[[[ 0.82207842  0.77224802  0.80734847 ...,  0.13750934  0.53164367
    0.95397311]
  [ 0.6913766   0.16888107  0.42421738 ...,  0.32027916  0.14697597
    0.70262951]
  [ 0.19146694  0.43920227  0.82708248 ...,  0.69575099  0.14051226
    0.70584554]
  ..., 
  [ 0.26979914  0.07588089  0.74032354 ...,  0.6739196   0.03651851
    0.44226776]
  [ 0.05232647  0.55924206  0.7499908  ...,  0.68651212  0.06501473
    0.71039408]
  [ 0.32976662  0.47015932  0.63889937 ...,  0.3509248   0.81882974
    0.79320657]]

 [[ 0.82180206  0.77203028  0.80718975 ...,  0.1375546   0.53134844
    0.95359512]
  [ 0.69097099  0.1691596   0.42431086 ...,  0.31994058  0.14725586
    0.70250188]
  [ 0.19175312  0.43931501  0.82658328 ...,  0.69521402  0.14095508
    0.70569048]
  ..., 
  [ 0.2695582   0.07634858  0.73969809 ...,  0.67375363  0.03686441
    0.44207974]
  [ 0.05274952  0.55886832  0.74986166 ...,  0.68628967  0.06541269
    0.70976269]
  [ 0.32963377  0.47027174  0.63886278 ...,  0.35089978  0

In [12]:
B = solve_backward(u0, v0, dx, dy, 1, tf, Du, Dv, lamb, tau, sigm, kapp) #Se queda pegado? Con dt = 2 tardá 24 min
print(B)

k=
0
k=
1
k=
2
k=
3
k=
4
k=
5
k=
6
k=
7
k=
8
[[[ 0.81559378  0.84763939  0.38493397 ...,  0.43561314  0.8729475
    0.40963664]
  [ 0.86026039  0.54041944  0.58807484 ...,  0.38776516  0.0841806
    0.61667162]
  [ 0.27172812  0.36750345  0.26259901 ...,  0.54117019  0.79570011
    0.34852872]
  ..., 
  [ 0.81546662  0.9430254   0.21462866 ...,  0.90731267  0.57892084
    0.28973465]
  [ 0.89630066  0.01960813  0.33672026 ...,  0.75186135  0.5199386
    0.08699742]
  [ 0.97038671  0.17028337  0.4526013  ...,  0.36282221  0.35190937
    0.32884615]]

 [[ 0.59048425  0.53264043  0.37312112 ...,  0.40290441  0.49738286
    0.44452077]
  [ 0.5606299   0.4508773   0.37836992 ...,  0.36583923  0.3231668
    0.46858658]
  [ 0.387572    0.30211073  0.25431956 ...,  0.39402651  0.48449576
    0.42845487]
  ..., 
  [ 0.59419286  0.56554405  0.36804035 ...,  0.6065624   0.48792086
    0.39198587]
  [ 0.54815541  0.29762758  0.33648272 ...,  0.55197921  0.43113916
    0.3066669 ]
  [ 0.58213395  0

In [ ]:
C = solve_mol(u0, v0, dx, dy, 2.5, tf, Du, Dv, lamb, tau, sigm, kapp,'RK2')
print(C)


In [ ]:
D = solve_mol(u0, v0, dx, dy, 2, tf, Du, Dv, lamb, tau, sigm, kapp,'RK4')
print(D)

In [35]:
animate_pattern(A)
#animate_pattern(B)


### c) Simulaciones con diferentes parametros

## Conclusiones

###TODO

## Referencias
<div id='ref1'\> [1] [IPython Notebook Documentation](http://ipython.readthedocs.org/en/stable/) IPython Documentation. *desc*. Revisado 16/12/2015
<div id='ref1'\> [2] [Python Documentation](https://docs.python.org/3/) Python Documentation. *desc*. Revisado 16/12/2015

## Anexos
Comenzamos cargando los módulos necesarios:

In [2]:
import numpy as np
import scipy as sp
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib.colors import cnames
from matplotlib import animation

#%matplotlib notebook
#%load_ext memory_profiler

<div id='an4'\> 
### Implementación de diferencias finitas

In [3]:
def fin_dif(N, x0, xN, y0, yN, ydN, yddN):
    if (N<4):
        print('Error: Se necesita al menos 4 intervalos (5 puntos).');
        return;
    #inicializando vector de y con ceros.
    y = np.zeros([N+1,1])
    #tamaño del intervalo
    h = (xN-x0)/N;
    x = np.linspace(x0, xN, N+1);
    
    #estableciendo valores conocidos
    y[0] = y0;
    y[N] = yN;
    y[N-1] = y[N]-ydN*h
    y[N-2] = yddN*h**4+yN-2*ydN*h;
    
    M = (N+1)-4;
    A = np.zeros([M, M]);
    b = np.zeros([M, 1]);
    
    #creando matriz A
    for i in range(M):
        if (i >= 1):
            A[i,i-1] = -1;
        A[i,i] = 4;
        if (i <= M-2):
            A[i,i+1] = -2*h**4-6;
        if (i <= M-3):
            A[i,i+2] = 4;
        if (i <= M-4):
            A[i,i+3] = -1;
            
    #creando vector b
    b[0] = y[0];
    if (M >= 3):
        b[M-3] = b[M-3] + y[N-2];
    if (M >= 2):
        b[M-2] = b[M-2] + y[N-1] - 4*y[N-2];
    b[M-1] = b[M-1] + y[N] - 4*y[N-1] - (-2*h**4-6)*y[N-2];
    
    #convirtiendo la matriz A en triangular superior
    for i in range(M-1):
        A[i+1, :] = A[i+1, :]+A[i, :]/A[i,i];
        b[i+1] = b[i+1]+b[i]/A[i,i];
    
    # Resolviendo el sistema de ecuaciones lineales con backward substitution
    for i in range(M-1,-1,-1):
        j = i+1;
        if (i== M-1):
            y[j] = b[i]/A[i,i]
        elif (i== M-2):
            y[j] = (b[i]-A[i,i+1]*y[j+1])/A[i,i];
        elif (i== M-3):
            y[j] = (b[i]-A[i,i+1]*y[j+1]-A[i,i+2]*y[j+2])/A[i,i];
        else:
            y[j] = (b[i]-A[i,i+1]*y[j+1]-A[i,i+2]*y[j+2]-A[i,i+3]*y[j+3])/A[i,i];
            
    #formateando output
    f = np.zeros([N+1, 2]);
    f[:,0] = x;
    f[:,1] = y[:,0];
    return f;

<div id='an5'\> 
### Utilidades

In [4]:
"""
> sol must be a 3D numpy array, with dimensions (Nt,Nx,Ny)
> time_step:  miliseconds to update canvas
> frame_step: Jump step for frames to show
"""
def animate_pattern(sol, time_step=10, frame_step=20):
    #time index
    t_ind = 0
    fig = plt.figure()
    im = plt.imshow(sol[t_ind], cmap=plt.cm.winter)
    plt.xticks([]); plt.yticks([])

    #setting the number of frames
    frames = int(sol.shape[0]/frame_step)
    
    #update function
    def updatefig(t_ind):
        im = plt.imshow(sol[t_ind*frame_step], cmap=plt.cm.winter)
        return im,
    
    #animate it!
    ani = animation.FuncAnimation(fig, updatefig, frames=frames, 
                    interval=time_step, blit=True, repeat=False)
    plt.show()